# Setup: El Dataset de Candidatos

In [26]:
import pandas as pd 
import numpy as np 

data = {
    'nombre': ['Ana', 'Beto', 'Carla', 'Daniel', 'Elena', 'Fer'],
    'edad': [25, 30, 22, 35, 28, 40],
    'ciudad': ['Bogotá', 'Medellín', 'Bogotá', 'Cali', 'Medellín', 'Cali'],
    'salario': [2500, 4500, 1800, 5000, 3200, 6000],
    'experiencia': [2, 5, 1, 8, 3, 12], # Años
    'lenguaje': ['Python', 'Java', 'Python', 'C#', 'Python', 'Go']
}

#Convertirlo a DataFrame
data_df=pd.DataFrame(data)
print(data_df)

   nombre  edad    ciudad  salario  experiencia lenguaje
0     Ana    25    Bogotá     2500            2   Python
1    Beto    30  Medellín     4500            5     Java
2   Carla    22    Bogotá     1800            1   Python
3  Daniel    35      Cali     5000            8       C#
4   Elena    28  Medellín     3200            3   Python
5     Fer    40      Cali     6000           12       Go


# Ejemplo 1: Anatomía de una Máscara (Paso a Paso)

Teoría: Entender que el filtro ocurre en dos fases: primero preguntas (True/False), luego filtras.

In [27]:
print("-------Fase 1: La Pregunta-----")
#Quien gana más de 3000
mascara_ask1=data_df['salario']>3000

print("Esto es la máscara (Solo True/False):")
print(mascara_ask1)
print(f'Tipo de dato:{mascara_ask1.dtype}')

print("\n-----FASE 2: La respuesta----")
#Aplicamos la máscara al DataFrame
resultado=data_df[mascara_ask1]
print(resultado)

-------Fase 1: La Pregunta-----
Esto es la máscara (Solo True/False):
0    False
1     True
2    False
3     True
4     True
5     True
Name: salario, dtype: bool
Tipo de dato:bool

-----FASE 2: La respuesta----
   nombre  edad    ciudad  salario  experiencia lenguaje
1    Beto    30  Medellín     4500            5     Java
3  Daniel    35      Cali     5000            8       C#
4   Elena    28  Medellín     3200            3   Python
5     Fer    40      Cali     6000           12       Go


Explicación línea por línea:

    mascara_ask1=data_df['salario']>3000: Aquí NO se filtra nada. Solo se crea una Serie de booleanos. Si miras el print, verás: False, True, False....

    data_df[mascara_ask1]: Aquí ocurre la magia. Pandas pone la máscara sobre el DataFrame original. Solo las filas que en la máscara decían True sobreviven. Las False desaparecen.

# Ejemplo 2: Lógica Combinada (&, |, ~)

Teoría: Usar operadores bitwise (Ampersand y Pipe) obligatoriamente con paréntesis.

In [28]:
print("--------- Filtrado Multi-Condición--------------")

#Queremos: Gente de 'Bogota' y que sepa 'Python'
#Error Común: Usar 'and' da error. Debes usar '&'
filtro_complejo=(data_df['ciudad']=='Bogotá') &(data_df['lenguaje']=='Python')

candidatos_ideales=data_df[filtro_complejo]
print(candidatos_ideales)

print("\n ----------- Filtrado cn Negación(NOT)----------")
#Queremos: Todos menos los de Cali
filtro_no_cali=~(data_df['ciudad']=='Cali')
#El simbolo ~ invierte:True se vuelve False

print(data_df[filtro_no_cali])

--------- Filtrado Multi-Condición--------------
  nombre  edad  ciudad  salario  experiencia lenguaje
0    Ana    25  Bogotá     2500            2   Python
2  Carla    22  Bogotá     1800            1   Python

 ----------- Filtrado cn Negación(NOT)----------
  nombre  edad    ciudad  salario  experiencia lenguaje
0    Ana    25    Bogotá     2500            2   Python
1   Beto    30  Medellín     4500            5     Java
2  Carla    22    Bogotá     1800            1   Python
4  Elena    28  Medellín     3200            3   Python


Explicación línea por línea:

    (condicion1) & (condicion2): Los paréntesis son vitales. Sin ellos, Python se confunde con el orden de las operaciones.

    ~: Es el operador de negación. Es útil para decir "Dame todo lo que NO sea esto".

# Ejemplo 3: Métodos Eficientes (isin, between)

Teoría: Simplificar el código. En lugar de escribir 5 "OR", usamos una lista.

In [29]:
print("------- Método .ISIN(La lista VIP)--------")
# Queremos gente de Bogotá O Cali.
# Forma larga (fea): (ciudad == 'Bogotá') | (ciudad == 'Cali')
# Forma profesional:
ciudades_target = ['Bogotá','Cali']
filtro_ciudades=data_df['ciudad'].isin(ciudades_target)

print(data_df[filtro_ciudades])

print("\n------- Método.BETWEEN(El Rango)-------")
# Queremos gente con edad entre 25 y 35 (inclusivo)
filtro_edad=data_df['edad'].between(25,35)
print(data_df[filtro_edad])

------- Método .ISIN(La lista VIP)--------
   nombre  edad  ciudad  salario  experiencia lenguaje
0     Ana    25  Bogotá     2500            2   Python
2   Carla    22  Bogotá     1800            1   Python
3  Daniel    35    Cali     5000            8       C#
5     Fer    40    Cali     6000           12       Go

------- Método.BETWEEN(El Rango)-------
   nombre  edad    ciudad  salario  experiencia lenguaje
0     Ana    25    Bogotá     2500            2   Python
1    Beto    30  Medellín     4500            5     Java
3  Daniel    35      Cali     5000            8       C#
4   Elena    28  Medellín     3200            3   Python


Explicación línea por línea:

    .isin(['A', 'B']): Verifica si el valor de la fila está dentro de la lista que le pasas. Es mucho más limpio que usar muchos operadores |.

    .between(25, 35): Es equivalente a escribir (edad >= 25) & (edad <= 35). Ahorra escritura y errores de paréntesis.

# Ejemplo 4: El Estilo SQL (.query)

Teoría: La técnica favorita de Matt Harrison. Escribir condiciones como si fuera texto natural o SQL.

In [30]:
print("----- Método .Query(Estilo Harrison)-----")
# Queremos: Salario mayor a 2000 Y (Experiencia > 3 O Lenguaje es Python)
# Escribir esto con corchetes y paréntesis sería un dolor de cabeza.

resultado_query=data_df.query("salario > 2000 and (experiencia > 3 or lenguaje == 'Python' )")
print(resultado_query)

----- Método .Query(Estilo Harrison)-----
   nombre  edad    ciudad  salario  experiencia lenguaje
0     Ana    25    Bogotá     2500            2   Python
1    Beto    30  Medellín     4500            5     Java
3  Daniel    35      Cali     5000            8       C#
4   Elena    28  Medellín     3200            3   Python
5     Fer    40      Cali     6000           12       Go


Explicación línea por línea:

    data_df.query("string"): Dentro de las comillas, puedes escribir lógica casi humana.

    Ventaja: Aquí SÍ puedes usar las palabras and y or. Pandas las traduce internamente.

    Legibilidad: Es mucho más fácil de leer para alguien que viene de SQL o Excel.